In [ ]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import make_scorer, f1_score, confusion_matrix, roc_curve, accuracy_score, auc, roc_auc_score, classification_report
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

In [ ]:

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [0.01, 0.1, 1]},
                    {'kernel': ['linear'], 'C': [0.01, 0.1, 1]},
                    {'kernel': ['poly'], 'C': [0.01, 0.1], 'degree': [3, 4], 'coef0': [0.1, 0.5]}]


In [ ]:
data = pd.read_csv('data/processed/mushrooms_pca.csv')
y = data['class']
X = data.drop('class', axis=1)

X_train, X_test, y_train, y_test = \
        train_test_split(X, y.ravel(), test_size=.1, random_state=42)

In [ ]:
grid_search = GridSearchCV(clf, tuned_parameters, scoring='f1',
                           cv=10, return_train_score=True, n_jobs=3)
grid_search.fit(X_train, y_train)

In [ ]:
res=pd.DataFrame(grid_search.cv_results_).sort_values('rank_test_score').reset_index(drop=True)
res = res[['param_kernel', 'param_coef0', 'param_C', 'mean_test_score', 'mean_fit_time']]
print(res.head(n=8).to_latex())

In [ ]:
pickle.dump(grid_search, open("models/svm.p", "wb" ) )
y_true, y_pred = y_test, grid_search.predict(X_test)
grid_search
print(grid_search.best_params_)
report = classification_report(y_true, y_pred)
print(report)